# StarRocks

>[StarRocks](https://www.starrocks.io/) 是一个高性能分析数据库。
`StarRocks` 是新一代的亚秒级 MPP 数据库，适用于全分析场景，包括多维分析、实时分析和即席查询。

>通常 `StarRocks` 被归类为 OLAP，并且在[ClickBench — 分析型数据库的基准测试](https://benchmark.clickhouse.com/)中表现出色。由于它拥有超快的向量化执行引擎，因此也可以用作快速的向量数据库。

在这里，我们将展示如何使用 StarRocks Vector Store。

## 设置

In [1]:
%pip install --upgrade --quiet  pymysql langchain-community

在开头设置 `update_vectordb = False`。如果没有文档更新，我们就不需要重新构建文档的 embeddings。

In [1]:
from langchain.chains import RetrievalQA
from langchain_community.document_loaders import (
    DirectoryLoader,
    UnstructuredMarkdownLoader,
)
from langchain_community.vectorstores import StarRocks
from langchain_community.vectorstores.starrocks import StarRocksSettings
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain_text_splitters import TokenTextSplitter

update_vectordb = False

/Users/dirlt/utils/py3env/lib/python3.9/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (5.1.0)/charset_normalizer (2.0.9) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


## 加载文档并将其分割成 token

加载 `docs` 目录下的所有 markdown 文件

对于 StarRocks 文档，你可以从 https://github.com/StarRocks/starrocks 克隆仓库，其中包含 `docs` 目录。

In [2]:
loader = DirectoryLoader(
    "./docs", glob="**/*.md", loader_cls=UnstructuredMarkdownLoader
)
documents = loader.load()

将文档拆分为 token，并将 `update_vectordb` 设置为 `True`，因为有新的文档/token。

In [3]:
# load text splitter and split docs into snippets of text
text_splitter = TokenTextSplitter(chunk_size=400, chunk_overlap=50)
split_docs = text_splitter.split_documents(documents)

# tell vectordb to update text embeddings
update_vectordb = True

In [4]:
split_docs[-20]

Document(page_content='Compile StarRocks with Docker\n\nThis topic describes how to compile StarRocks using Docker.\n\nOverview\n\nStarRocks provides development environment images for both Ubuntu 22.04 and CentOS 7.9. With the image, you can launch a Docker container and compile StarRocks in the container.\n\nStarRocks version and DEV ENV image\n\nDifferent branches of StarRocks correspond to different development environment images provided on StarRocks Docker Hub.\n\nFor Ubuntu 22.04:\n\n| Branch name | Image name              |\n  | --------------- | ----------------------------------- |\n  | main            | starrocks/dev-env-ubuntu:latest     |\n  | branch-3.0      | starrocks/dev-env-ubuntu:3.0-latest |\n  | branch-2.5      | starrocks/dev-env-ubuntu:2.5-latest |\n\nFor CentOS 7.9:\n\n| Branch name | Image name                       |\n  | --------------- | ------------------------------------ |\n  | main            | starrocks/dev-env-centos7:latest     |\n  | branch-3.0      

In [5]:
print("# docs  = %d, # splits = %d" % (len(documents), len(split_docs)))

# docs  = 657, # splits = 2802


## 创建 vectordb 实例

### 将 StarRocks 用作向量数据库

In [6]:
def gen_starrocks(update_vectordb, embeddings, settings):
    if update_vectordb:
        docsearch = StarRocks.from_documents(split_docs, embeddings, config=settings)
    else:
        docsearch = StarRocks(embeddings, settings)
    return docsearch

## 将 Token 转换为嵌入并放入向量数据库

这里我们使用 StarRocks 作为向量数据库，你可以通过 `StarRocksSettings` 来配置 StarRocks 实例。

配置 StarRocks 实例与配置 MySQL 实例非常相似。你需要指定：
1. host/port
2. 用户名（默认：'root'）
3. 密码（默认：''）
4. 数据库（默认：'default'）
5. 表（默认：'langchain'）

In [8]:
embeddings = OpenAIEmbeddings()

# configure starrocks settings(host/port/user/pw/db)
settings = StarRocksSettings()
settings.port = 41003
settings.host = "127.0.0.1"
settings.username = "root"
settings.password = ""
settings.database = "zya"
docsearch = gen_starrocks(update_vectordb, embeddings, settings)

print(docsearch)

update_vectordb = False

Inserting data...: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2802/2802 [02:26<00:00, 19.11it/s]


zya.langchain @ 127.0.0.1:41003

username: root

Table Schema:
----------------------------------------------------------------------------
|name                    |type                    |key                     |
----------------------------------------------------------------------------
|id                      |varchar(65533)          |true                    |
|document                |varchar(65533)          |false                   |
|embedding               |array<float>            |false                   |
|metadata                |varchar(65533)          |false                   |
----------------------------------------------------------------------------



## 构建 QA 并向其提问

In [10]:
llm = OpenAI()
qa = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=docsearch.as_retriever()
)
query = "is profile enabled by default? if not, how to enable profile?"
resp = qa.run(query)
print(resp)

 No, profile is not enabled by default. To enable profile, set the variable `enable_profile` to `true` using the command `set enable_profile = true;`
